In [5]:
import chess
from tqdm import tqdm
from project import Searcher, Position, initial, sunfish_move, sunfish_move_to_str

In [2]:
hist = [Position(initial, 0, (True, True), (True, True), 0, 0)]
time_limit = 1.2  # Seconds, Could do a node limit??
move, info = sunfish_move(Searcher(), hist, time_limit=time_limit,)

print(f'Move: {sunfish_move_to_str(move)}, Info: {info}')

Move: g1f3, Info: {'depth': 6, 'gamma': 20, 'score': 19, 'nodes': 30416}


In [3]:
hist[0].move(move)

Position(board='\n         \n         \nr.bkqbnr \npppppppp \n..n..... \n........ \n........ \n........ \nPPPPPPPP \nRNBKQBNR \n         \n         ', score=-37, wc=(True, True), bc=(True, True), ep=0, kp=0)

In [4]:
[el for el in hist[0].gen_moves()]

[Move(i=81, j=71, prom=''),
 Move(i=81, j=61, prom=''),
 Move(i=82, j=72, prom=''),
 Move(i=82, j=62, prom=''),
 Move(i=83, j=73, prom=''),
 Move(i=83, j=63, prom=''),
 Move(i=84, j=74, prom=''),
 Move(i=84, j=64, prom=''),
 Move(i=85, j=75, prom=''),
 Move(i=85, j=65, prom=''),
 Move(i=86, j=76, prom=''),
 Move(i=86, j=66, prom=''),
 Move(i=87, j=77, prom=''),
 Move(i=87, j=67, prom=''),
 Move(i=88, j=78, prom=''),
 Move(i=88, j=68, prom=''),
 Move(i=92, j=73, prom=''),
 Move(i=92, j=71, prom=''),
 Move(i=97, j=78, prom=''),
 Move(i=97, j=76, prom='')]

In [6]:
def play_on(board, n):
    from copy import copy
    board0 = copy(board)
    for i in tqdm(range(n)):
        moves = [el for el in board.generate_legal_moves()]
        if len(moves):
            board.push(moves[-1])
        else:
            board = board0


In [7]:
n = int(2e5)
play_on(chess.Board(), n)

100%|██████████| 200000/200000 [00:14<00:00, 13566.52it/s]


In [8]:
play_on(hist[0], n)

100%|██████████| 200000/200000 [00:10<00:00, 19054.06it/s]


In [22]:
import copy

In [30]:
pst = {
    'P': (   0,   0,   0,   0,   0,   0,   0,   0,
            78,  83,  86,  73, 102,  82,  85,  90,
             7,  29,  21,  44,  40,  31,  44,   7,
           -17,  16,  -2,  15,  14,   0,  15, -13,
           -26,   3,  10,   9,   6,   1,   0, -23,
           -22,   9,   5, -11, -10,  -2,   3, -19,
           -31,   8,  -7, -37, -36, -14,   3, -31,
             0,   0,   0,   0,   0,   0,   0,   0),
    'N': ( -66, -53, -75, -75, -10, -55, -58, -70,
            -3,  -6, 100, -36,   4,  62,  -4, -14,
            10,  67,   1,  74,  73,  27,  62,  -2,
            24,  24,  45,  37,  33,  41,  25,  17,
            -1,   5,  31,  21,  22,  35,   2,   0,
           -18,  10,  13,  22,  18,  15,  11, -14,
           -23, -15,   2,   0,   2,   0, -23, -20,
           -74, -23, -26, -24, -19, -35, -22, -69),
    'B': ( -59, -78, -82, -76, -23,-107, -37, -50,
           -11,  20,  35, -42, -39,  31,   2, -22,
            -9,  39, -32,  41,  52, -10,  28, -14,
            25,  17,  20,  34,  26,  25,  15,  10,
            13,  10,  17,  23,  17,  16,   0,   7,
            14,  25,  24,  15,   8,  25,  20,  15,
            19,  20,  11,   6,   7,   6,  20,  16,
            -7,   2, -15, -12, -14, -15, -10, -10),
    'R': (  35,  29,  33,   4,  37,  33,  56,  50,
            55,  29,  56,  67,  55,  62,  34,  60,
            19,  35,  28,  33,  45,  27,  25,  15,
             0,   5,  16,  13,  18,  -4,  -9,  -6,
           -28, -35, -16, -21, -13, -29, -46, -30,
           -42, -28, -42, -25, -25, -35, -26, -46,
           -53, -38, -31, -26, -29, -43, -44, -53,
           -30, -24, -18,   5,  -2, -18, -31, -32),
    'Q': (   6,   1,  -8,-104,  69,  24,  88,  26,
            14,  32,  60, -10,  20,  76,  57,  24,
            -2,  43,  32,  60,  72,  63,  43,   2,
             1, -16,  22,  17,  25,  20, -13,  -6,
           -14, -15,  -2,  -5,  -1, -10, -20, -22,
           -30,  -6, -13, -11, -16, -11, -16, -27,
           -36, -18,   0, -19, -15, -15, -21, -38,
           -39, -30, -31, -13, -31, -36, -34, -42),
    'K': (   4,  54,  47, -99, -99,  60,  83, -62,
           -32,  10,  55,  56,  56,  55,  10,   3,
           -62,  12, -57,  44, -67,  28,  37, -31,
           -55,  50,  11,  -4, -19,  13,   0, -49,
           -55, -43, -52, -28, -51, -47,  -8, -50,
           -47, -42, -43, -79, -64, -32, -29, -32,
            -4,   3, -14, -50, -57, -18,  13,   4,
            17,  30,  -3, -14,   6,  -1,  40,  18),
}
# for black
pst_ = copy.deepcopy(pst)
for piece in pst_.keys():
    piece_array = -np.array(pst[piece]).reshape((8,8))[np.arange(7,-1,-1)]
    pst[piece.lower()] = tuple(piece_array.reshape((-1)))

In [31]:
pst['k']

(-17,
 -30,
 3,
 14,
 -6,
 1,
 -40,
 -18,
 4,
 -3,
 14,
 50,
 57,
 18,
 -13,
 -4,
 47,
 42,
 43,
 79,
 64,
 32,
 29,
 32,
 55,
 43,
 52,
 28,
 51,
 47,
 8,
 50,
 55,
 -50,
 -11,
 4,
 19,
 -13,
 0,
 49,
 62,
 -12,
 57,
 -44,
 67,
 -28,
 -37,
 31,
 32,
 -10,
 -55,
 -56,
 -56,
 -55,
 -10,
 -3,
 -4,
 -54,
 -47,
 99,
 99,
 -60,
 -83,
 62)

In [33]:
s = str(chess.Board()).replace(' ','').replace('\n','')
eval = 0
for i, char in enumerate(s):
    if char == '.':
        continue
    else:
        eval += pst[char][i]
print(eval)

0


In [20]:
import numpy as np
np.array(pst['K']).reshape((8,8))[np.arange(7,-1,-1)].reshape((-1))

array([ 17,  30,  -3, -14,   6,  -1,  40,  18,  -4,   3, -14, -50, -57,
       -18,  13,   4, -47, -42, -43, -79, -64, -32, -29, -32, -55, -43,
       -52, -28, -51, -47,  -8, -50, -55,  50,  11,  -4, -19,  13,   0,
       -49, -62,  12, -57,  44, -67,  28,  37, -31, -32,  10,  55,  56,
        56,  55,  10,   3,   4,  54,  47, -99, -99,  60,  83, -62])